# Formulas
## by Decla

# Content（可直接 ctrl+F 相关索引）
## import is important！
## 抓取股票
> 1（我承认这里有define function）\
> 2（但这里没有自己define的）
## 双均线策略，以 20 天和 60 天为例
## MDD
## 最大回撤期
## 设置一个rolling期（以41天为例）以及一个上下波动的阈值（以4为例）
>1.单纯画图\
>2.画出买入卖出点\
>3.对比strategy return和原股票return（我认为大部分算return都可以按这个来）
## sharpe ratio
## Compound Annual Growth Rate，CAGR
## mean-reversion: 抓取股票后由一个rolling 以及一个阈值（这里是两倍std）确定的上下限，制定策略
> 1.画图\
> 2.计算sharpe ratio
## 用之前的一定天数（以 3 天为例）预测明天的log return
## MACD策略，长期(26天)短期(12天)差值与MACD signal(9天)
## RSI 
## Combine MACD and RSI
## PCA
> group8\
> group9
## Cluster

## <font color = Red> btw, if u think changing the name of dataframes is time-consuming, let's remind the function .copy() 

## import is important！

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from pandas_datareader import data as pdr

import datetime
import random
import yfinance as yf
import pandas_datareader.data as web 

(
    yf
    .pdr_override()
)
np.set_printoptions(precision = 3)

plt.style.use("ggplot")

mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.color"] = "grey"
mpl.rcParams["grid.alpha"] = 0.25

mpl.rcParams["axes.facecolor"] = "white"

mpl.rcParams["legend.fontsize"] = 14

from pandas.plotting import scatter_matrix
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

from sklearn.decomposition import TruncatedSVD

from numpy.linalg import inv, eig, svd

from sklearn.manifold import TSNE

from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold

import matplotlib.ticker as ticker
from itertools import cycle

## 抓取股票
### 1

In [ ]:
stocks =\
    ["WMT", # Note that FB has been changed to META
     "IBM"]
def extract_sp(stocks, start, end):
    def data(ticker):
        return(pdr.get_data_yahoo(ticker,
                                  start = start,
                                  end = end)
              )
    FAANG_Stock = map(data, stocks)
    return(pd.concat(FAANG_Stock,
                     keys = stocks,
                     names = ["Tickers", "Date"]
                    )
          )
#设置起始和结束时间
FAANG =\
    extract_sp(stocks,
               datetime.datetime(2014, 10, 13),
               datetime.datetime(2022, 9, 14)
              )
Daily_Closing_Price =\
(
FAANG[["Adj Close"]]
    .reset_index()
    .pivot(index = "Date",
           columns = "Tickers",
           values = "Adj Close")
)
Daily_Closing_Price

### 2（这里没有定义函数）

In [ ]:
google =\
(    web
     .get_data_yahoo("GOOGL", 
                     start, 
                     end)
)

## 双均线策略，以 20 天和 60 天为例
>注意只是画了红色的矩形图，

In [ ]:
FAANG2 =Daily_Closing_Price["IBM"].copy()
FAANG2 = pd.DataFrame(FAANG2)

#更改天数在这里
FAANG2["SMA_20"] =\
(
    FAANG2
    ["IBM"]
    .rolling(window = 20)
    .mean()
)
FAANG2["SMA_60"] =\
(
    FAANG2
    ["IBM"]
    .rolling(60)
    .mean()
)

#确定position
FAANG2["positions"] =\
(
    np
    .where(FAANG2["SMA_20"] > FAANG2["SMA_60"],
           1, -1)
)
FAANG2 =\
(
    FAANG2
    .dropna()
)

ax =\
(
    FAANG2
    [["IBM", 
      "SMA_20", 
      "SMA_60", 
      "positions"]]
    .plot(secondary_y = "positions",
          color = ["grey", 
                   "blue",
                   "green",
                   "red"],
          style = ["-",
                  "--",
                  "--",
                  "-"],
          figsize = [18, 6]
         )
)

(
    ax
    .legend(loc = "upper center",
            shadow = True,
            ncol = 4,
            bbox_to_anchor = (0.55, 1.10),
            fancybox = True
           )
)

## MDD

In [ ]:
FAANG2["log_returns"] =\
(
    np
    .log(FAANG2["Adj Close"] / FAANG2["Adj Close"].shift(1)
        )
)
FAANG2["strategy_returns"] =\
(
    FAANG2["positions"]
    .shift(1)
    *
    FAANG2["log_returns"] # passive following
)
FAANG2["cumulative_returns"] =\
(
    FAANG2["strategy_returns"]
    .cumsum()
    .apply(np.exp)
)
FAANG2["max_gross_performance"] =\
(
    FAANG2
    ["cumulative_returns"]
    .cummax()
)

# (
#     FAANG2
#     [["cumulative_returns", "max_gross_performance"]]
#     .dropna()
#     .plot(figsize = [18, 8]
#          )
# )
FAANG2['drawdown'] = (FAANG2["max_gross_performance"] - FAANG2["cumulative_returns"])/FAANG2["max_gross_performance"]
FAANG2['drawdown'].max()

## 最大回撤期
## <font color = green> for instance, find the `five` worst drawdown periods over the investment horizon and provide their net drawdown in % and duration, respectively.

In [ ]:
stock_drawdown = (
    pd.concat(
        [
            FAANG2[FAANG2.drawdown==0],# we need to get drawdown == 0
            FAANG2[-1:] #plus last one
        ],
        axis=0
    )
)
lst_day = []
lst_drawdown = []
for i in range(len(stock_drawdown) - 1):
    days = (stock_drawdown.index[i+1] - stock_drawdown.index[i]).days
    max_drawdown = FAANG2[stock_drawdown.index[i]:stock_drawdown.index[i+1]]["drawdown"].max()
    lst_day.append(days)
    lst_drawdown.append(max_drawdown)

drawdown = pd.DataFrame([lst_day,lst_drawdown]).T
drawdown.columns = ["days", "drawdown"]    
drawdown.loc[drawdown["drawdown"].nlargest(5).index]#这里改数字

## 设置一个rolling期（以41天为例）以及一个上下波动的阈值（以4为例）
### 1.单纯画图

In [ ]:
FAANG5 =Daily_Closing_Price["WMT"].copy()
FAANG5 = pd.DataFrame(FAANG5)

#更改天数在这里
FAANG5["SMA_41"] =\
(
    FAANG5["WMT"]
    .rolling(window = 41)
    .mean()
)

#更改阈值在这里
threshold = 4

FAANG5["distance"] = FAANG5["WMT"] - FAANG5["SMA_41"]

(FAANG5["distance"]
 .dropna()
 .plot(figsize = [16, 6]
      )
)

plt.axhline(threshold, color = "blue", ls = "--")

plt.axhline(0, color = "green", ls = "--")

plt.axhline(-threshold, color = "red", ls = "--")


### 2.画出买入卖出点

In [ ]:
FAANG6 =FAANG5.copy()
FAANG6=FAANG6.reset_index()
(
FAANG5["distance"]
.dropna()
.plot(figsize = [16, 7],
      color = "grey",
      alpha = 0.80)
)

# Upper-bound threshold

plt.axhline(4,
            color = "blue",
            ls = "--")

# Lower-bound threshold

plt.axhline(-4,
            color = "blue",
            ls = "--")

# Sell Signal

FAANG5["positions"] =\
(
    np
    .where(FAANG5["distance"] > 4,
           -1, np.nan)
)

# Buy Signal

FAANG5["positions"] =\
(
    np
    .where(FAANG5["distance"] < -4,
           1, FAANG5["positions"])
)

# Market-Neutral Signal

FAANG5["positions"] =\
(
    np
    .where(FAANG5["distance"] * FAANG5["distance"].shift(1) < 0,
           0, FAANG5["positions"])
)
            
sum_positions = 0

for i in range(1994):
    if FAANG5["positions"][i] == 1:
        plt.scatter(FAANG6['Date'][i],1,color='red',marker='.',s=100)
        sum_positions  +=1
    elif FAANG5["positions"][i] == -1:
        plt.scatter(FAANG6['Date'][i],-1,color='blue',marker='.',s=100)
        sum_positions  +=1
    i +=1

### 3.对比strategy return和原股票return（我认为大部分算return都可以按这个来）

In [ ]:
FAANG5["RETURNS"] =\
(
    np
    .log(FAANG5["WMT"]
         /
         FAANG5["WMT"].shift(1)
        )
)
FAANG5["positions"] =\
    (FAANG5["positions"]
     .ffill()
    )
FAANG5["STRATEGY"] =\
    (
    FAANG5
    ["positions"]
    .shift(1)
    *
    FAANG5["RETURNS"]
    )
(
    FAANG5
    [["RETURNS", "STRATEGY"]]
    .dropna()
    .cumsum()
    .apply(np.exp)
    .plot(figsize = [18, 9]
         )
)

## sharpe ratio

In [ ]:
Portfolio["returns"] = Portfolio["total"].pct_change()

Sharpe =np.sqrt(253) * (Portfolio["returns"].mean() / Portfolio["returns"].std())

## Compound Annual Growth Rate，CAGR

In [ ]:
#计算交易日天数
days_I =\
(    (FAANG2.index[-1] - FAANG2.index[0])
     .days
)

FAANG2 =\
(
    FAANG2
    .dropna()
)

CAGR_I =\
(
    (
      (
        (FAANG2["strategy_returns"][-1]+1) / (FAANG2["strategy_returns"][0]+1)
      )
        **(365.0/days_I) 
    ) 
    - 1
)

CAGR_I 

## mean-reversion
## 抓取股票后由一个rolling（42天） 以及一个阈值（这里是两倍std）确定的上下限，制定策略
### 1.画图

In [ ]:
google["Log_return"] = np.log(google["Close"]/google["Close"].shift(1))

#更改rolling天数在这一段！一段！一段！
google["SMA_42"] = google["Close"].rolling(window=42).mean()
google["Distance"] = google["Close"] - google["SMA_42"]
google["Upper_threshold"] = 2*google["Close"].rolling(window=42).std()
google["Lower_threshold"] = -2*google["Close"].rolling(window=42).std()

google["Trading_position"] = np.where(google["Distance"] > google["Upper_threshold"], -1, np.nan)
google["Trading_position"] = np.where(google["Distance"] < google["Lower_threshold"], 1, google["Trading_position"])
google["Trading_position"] = np.where(google["Distance"]*google["Distance"].shift(1) < 0, 0, google["Trading_position"])
google["Trading_position"].ffill(inplace=True)

google1 = google.copy()
mask = (google1["Trading_position"]!=google1["Trading_position"].shift(1))
google1["Signal"] = google1["Trading_position"].where(mask, 0)

plt.figure(figsize=(18,9))
plt.plot(google1["Close"], label="Close")
plt.plot(google1["SMA_42"], label="SMA_42")
plt.plot(google1["SMA_42"]+google1["Upper_threshold"], label="Upper_threshold")
plt.plot(google1["SMA_42"]+google1["Lower_threshold"], label="Lower_threshold")
plt.plot(google1[google1["Signal"]==1]["Close"],"g^")
plt.plot(google1[google1["Signal"]==-1]["Close"],"rv")
plt.legend()
plt.show()

### 2.计算Sharpe ratio

In [ ]:
google1["Log_strategy"] = google1["Log_return"]*google1["Trading_position"].shift(1)
google1["Strategy"] = np.exp(google1["Log_strategy"])-1
sharpe_ratio = np.sqrt(253)*np.mean(google1["Strategy"])/np.std(google1["Strategy"])
sharpe_ratio

## 用之前的一定天数（以 3 天为例）预测明天的log return

In [ ]:
ge = pdr.get_data_yahoo("GE",dt.datetime(2011, 8, 1), dt.datetime(2020, 8, 1))
ge["Log_return"] = np.log(ge["Close"]/ge["Close"].shift(1))

#更改天数在这里
lags = 3
cols = []
for lag in range(1,lags+1):
    col = f"lag_{lag}"
    cols.append(col)
    ge[col] = ge["Log_return"].shift(lag)
ge1 = ge.copy()
ge1.dropna(inplace=True)
OLS = np.linalg.lstsq(ge1[cols],ge1["Log_return"])[0]
ge1["Prediction"] = np.dot(ge1[cols], OLS)

accuracy = np.sign(ge1["Prediction"]*ge1["Log_return"]).value_counts()
accuracy.values[0]/sum(accuracy)

#画图
# plt.figure(figsize=(18,9))
# plt.plot(ge1["Prediction"].cumsum(), label="Prediction_1_2_3")
# plt.legend()
# plt.show()

#计算Sharpe ratio
#np.sqrt(253)*np.mean((np.exp(ge1["Prediction"])-1))/np.std((np.exp(ge1["Prediction"])-1))

## MACD策略，长期(26天)短期(12天)差值与MACD signal(9天)

In [ ]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2017, 12, 31)
AAPL =\
(    web
     .get_data_yahoo("AAPL", 
                     start, 
                     end)
)
AAPL =AAPL.rename(columns={'Adj Close': 'Adj_Close'})
AAPL["mean_12"] =\
    (
    AAPL[["Adj_Close"]]
    .rolling(12)
    .mean()
    )
AAPL["mean_26"] =\
    (
    AAPL[["Adj_Close"]]
    .rolling(26)
    .mean()
    )
AAPL["MACD"]=AAPL["mean_12"]-AAPL["mean_26"]
AAPL["MACD_signal"] =\
    (
    AAPL[["MACD"]]
    .rolling(9)
    .mean()
    )
AAPL["Trading_Positions"] =\
    (
    np
    .where(AAPL["MACD"] > AAPL["MACD_signal"],
           1, -1)
    )

AAPL["Positions"] =\
(    AAPL["Trading_Positions"]
     .diff()
)

AAPL = AAPL.reset_index()
AAPL.set_index('Date', inplace=True)

fig =\
    (plt
     .figure(figsize = [16, 10]
            )
    )

sub =\
(    fig
    .add_subplot(111,
                 ylabel = "Stock Price")
)

(AAPL["Adj_Close"]
 .plot(ax = sub,
       color = "grey",
       lw = 0.80) # This is for closing price
)

(AAPL[["MACD_signal", 
              "MACD"]]
            .plot(ax = sub,
                  style = ["--", "--"],
                  lw = 0.80
                 )
)


# Buy

(
    sub
    .plot(AAPL.loc[AAPL['Positions'] == 2].index,#.astype(str),
          AAPL.Adj_Close[AAPL['Positions'] == 2],
          "g^",
          color = "green",
          markersize = 12)

)

# Sell

(
    sub
    .plot(AAPL.loc[AAPL['Positions'] == -2].index,#,
          AAPL.Adj_Close[AAPL['Positions'] == -2],
          "rv",
          color = "red",
          markersize = 12)
)

## RSI
<left> $ RS = \frac{AverageGain}{AverageLoss} $
    
<left> $ RSI = 100 - \frac{100}{1 + RS} $

In [ ]:
delta = df['Adj Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

ag = gain.rolling(window=window_RS).mean()
al = loss.rolling(window=window_RS).mean()

rs = ag / al
df['RSI'] = 100 - (100 / (1 + rs))

df["position"] = np.where(df['RSI'] > threshold_RSI, -1, 1)

df["position"] = df["position"].shift(1)
df["position"].fillna(0, inplace=True)

df["signal"] = df["position"].diff() 
df["signal"].fillna(0, inplace=True)
    
df["signal"]=np.sign(df["signal"])

# Create a figure and axis for the price chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the price data
ax.plot(df['Adj Close'], label='Price', color='b', linewidth=0.5)

# Plot buy positions (green) and sell positions (red) based on the 'Signal' column
buy_positions =df[df['signal']>0]
sell_positions = df[df['signal'] <0]

ax.scatter(buy_positions.index, buy_positions['Adj Close'], marker='^', color='g', label='Buy', s=10)
ax.scatter(sell_positions.index, sell_positions['Adj Close'], marker='v', color='r', label='Sell', s=10)

# Add labels and legend
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.set_title('RSI Buy and Sell Positions')
ax.legend()

## Combine MACD and RSI

In [ ]:
COM = pd.concat([MACD[['Adj Close', "position"]], RSI[["position"]]], axis = 1) 
COM.columns = ['Adj Close', 'position1', 'position2'] 
COM['position'] = np.where(COM['position1']*COM['position2']>0, COM['position1'], 0)
COM['signal'] = COM['position'].diff() 
COM['signal'].fillna(0, inplace=True)

# Create a figure and axis for the price chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the price data
ax.plot(df['Adj Close'], label='Price', color='b', linewidth=0.5)

# Plot buy positions (green) and sell positions (red) based on the 'Signal' column
buy_positions =df[df['signal'] > 0]
sell_positions = df[df['signal'] < 0]

ax.scatter(buy_positions.index, buy_positions['Adj Close'], marker='^', color='g', label='Buy', s=10)
ax.scatter(sell_positions.index, sell_positions['Adj Close'], marker='v', color='r', label='Sell', s=10)

# Add labels and legend
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.set_title('Combined Buy and Sell Positions')
ax.legend()

## PCA
### <font color = green> solution by group8 

In [ ]:
list_of_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
selected_symbols = random.sample(list_of_tickers['Symbol'].tolist(), 40)

np.random.seed(627)
selected_tickers = list_of_tickers.sample(n=40)
data = []
for ticker in selected_tickers['Symbol']:
    stock_data = yf.download(ticker, start="2009-01-01", end="2019-12-31")
    stock_data['Ticker'] = ticker
    data.append(stock_data['Adj Close'])
    
data = pd.concat(data, axis=1)
data.columns = selected_tickers['Symbol']

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Exploratory Data Analysis (EDA)
#热力图
corr = data.corr()
plt.figure(figsize = [16, 16]
          )
plt.title("A Heatmap for Correlation Matrix")
sns \
    .heatmap(corr,
             annot = True,
             cmap = "viridis")

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Data Transformation
missing_values =\
(
    data
    .isnull() # True (1) vs. False (0)
    .mean()
    .sort_values(ascending = False)
)
# missing_values.head(10)
drop_list =\
(
    sorted(list(missing_values[missing_values > 0.00]
                .index)
          )
)
stocks_df_cleaned =\
(
    data
    .drop(labels = drop_list,
          axis = 1)
)
# print(data.shape)
# print(stocks_df_cleaned.shape)

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Calculating Linear Daily Return
Daily_Linear_Return =\
(
    stocks_df_cleaned
    .pct_change(1)
)
Daily_Linear_Return =\
(
    Daily_Linear_Return[Daily_Linear_Return 
                        .apply(lambda x:(x - x.mean()
                                        ).abs() < (3 * x.std()
                                                  )
                              )
                        .all(1)
    ]
)

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Important considerations in data transformation for PCA
scaler =\
(
    StandardScaler()
    .fit(Daily_Linear_Return)
)
scaled_df =\
(
    pd
    .DataFrame(scaler.fit_transform(Daily_Linear_Return),
               columns = Daily_Linear_Return.columns,
               index = Daily_Linear_Return.index)
)
plt.figure(figsize = [16, 6]
          )

plt.title("WYNN Return")

plt.ylabel("Linear Return")

(
    scaled_df
    ["WYNN"]
    .plot()
)

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Data split
prop =\
    int(len(scaled_df) * 0.80)

X_Train = scaled_df[    : prop] # First 80% of the data
X_Test  = scaled_df[prop:     ] # Remaining 20% of the data

X_Train_Raw = Daily_Linear_Return[    :prop]
X_Test_Raw  = Daily_Linear_Return[prop:    ]

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Model Fitting
pca = PCA()
PrincipalComponent = pca.fit(X_Train)
pca.components_[0]

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Explained Variance
NumEigenValues = 10
fig, axes =\
(
    plt
    .subplots(ncols = 2,
              figsize = [16, 6]
             )
)
# Plot on the left panel
Series1 =\
(
    pd
    .Series(pca
            .explained_variance_ratio_[ :NumEigenValues]
           )
    .sort_values()
    * 100
)
# Plot on the right panel
Series2 =\
(
    pd
    .Series(pca
            .explained_variance_ratio_[ :NumEigenValues]
           )
    .cumsum()
    * 100
)
(
    Series1
    .plot
    .barh(ylim = (0, 9),
          title = "Explained Variance Ratio by Top 10",
          ax = axes[0]
         )
)
(
    Series2
    .plot(ylim = (0, 100),
          xlim = (0, 10),
          title = "Cumulative Explained Variance by Each PC",
          ax = axes[1]
         )
)
##分段---------------------------------------------------------------------------------------------------------------------------------##
(
    pd
    .Series(np
           .cumsum(pca
                   .explained_variance_ratio_)
           )
    .to_frame("Explained Variance")
    .head(NumEigenValues)
    .style
    .format("{:,.2%}".format)
)

##分段---------------------------------------------------------------------------------------------------------------------------------##
# Portfolio Weights
pca.components_
weights = pd.DataFrame()
for i in range(len(pca.components_)):
    weights["weights_{}".format(i)] = pca.components_[i] / sum(pca.components_[i])
    weights = weights.values.T
# Set the number of principal components to be considered
NumComponents = 4

# Extract the top principal components from the PCA object
# and create a DataFrame with columns named after the original features

topPortfolios =\
(
    pd
    .DataFrame(pca.components_[ : NumComponents],
               columns = stocks_df_cleaned.columns)
)

# Normalize the weights of the top portfolios such that the weights sum up to 1 for each portfolio
# This is done by dividing each weight by the sum of weights for the respective portfolio

eigen_portfolios =\
(
    topPortfolios
    .div(topPortfolios.sum(1),
         axis = 0)
)

# Rename the index of the eigen_portfolios DataFrame for better readability

eigen_portfolios.index = [f"Portfolio {i}" for i in range(NumComponents)]
                         
# Calculate the square root of the explained variance for each component
# This provides the standard deviation of returns for each eigenportfolio
np.sqrt(pca.explained_variance_)

##分段---------------------------------------------------------------------------------------------------------------------------------##
eigen_portfolios
                          
##分段---------------------------------------------------------------------------------------------------------------------------------##
(
    eigen_portfolios
    .T  # Transpose the DataFrame to have portfolios as columns and assets as rows
    .plot
    .bar(subplots = True,
         layout = (int(NumComponents), 1),
         legend = False,
         sharey = True,
         figsize = [16, 20],
         ylim = [-1, 1]
        )
)

plt.figure(figsize = [16, 6]
          )

sns.heatmap(topPortfolios,
            cmap = "viridis")

##分段---------------------------------------------------------------------------------------------------------------------------------##
#Best Eigen Portfolio
def calculate_sharpe_ratio(ts_returns, periods_per_year = 252):

    n_years = ts_returns.shape[0] / periods_per_year

    annualized_return = np.power(np.prod(1 + ts_returns), (1 / n_years)
                                ) - 1

    annualized_vol = ts_returns.std() * np.sqrt(periods_per_year)

    annualized_sharpe = annualized_return / annualized_vol

    return annualized_return, annualized_vol, annualized_sharpe

def recommend_optimal_portfolio():

    # Number of eigenportfolios or principal components
    
    n_portfolios = len(pca.components_)

    # Initialize arrays for annualized return, volatility, and Sharpe ratio of each eigenportfolio
    
    annualized_ret = np.array([0.] * n_portfolios)

    sharpe_metric = np.array([0.] * n_portfolios)

    annualized_vol = np.array([0.] * n_portfolios)

    # Variable to track the index of the eigenportfolio with the highest Sharpe ratio
    
    highest_sharpe = 0

    # Extract stock tickers from the scaled data
    
    stock_tickers =\
    (scaled_df
     .columns 
     .values)

    n_tickers = len(stock_tickers)

    # Extract principal components
    
    PCs = pca.components_

    # Loop through each eigenportfolio

    for i in range(n_portfolios):

        # Normalize the weights of the i-th eigenportfolio
        
        pc_w = PCs[i] / sum(PCs[i])

        # Create a DataFrame for the eigenportfolio weights
        
        eigen_prtfi =\
            (
                pd
                .DataFrame(data = {"weights": pc_w.squeeze() * 100},
                           index = stock_tickers)
            )

        # Calculate returns for the eigenportfolio
        
        eigen_prtfi.sort_values(by = ["weights"],
                                ascending = False,
                                inplace = True)

        eigen_prti_returns =\
            (
                np
                .dot(X_Train_Raw.loc[ : , eigen_prtfi.index],
                     pc_w)
            )

        eigen_prti_returns =\
            (
                pd
                .Series(eigen_prti_returns.squeeze(),
                        index = X_Train_Raw.index)
            )

        # Calculate annualized return, volatility, and Sharpe ratio for the eigenportfolio
        
        er, vol, sharpe = calculate_sharpe_ratio(eigen_prti_returns)

        # Store the metrics in their respective arrays
        
        annualized_ret[i] = er
        annualized_vol[i] = vol
        sharpe_metric[i] = sharpe

        # Replace NaN values in Sharpe metric array with zeros
        
        sharpe_metric = np.nan_to_num(sharpe_metric)

    # Let's find a portfolio with the HIGHEST Sharpe Ratio

    highest_sharpe = np.argmax(sharpe_metric)

    # Print the details of the eigenportfolio with the highest Sharpe ratio
    
    print("Our Eigen Portfolio #%d with the highest Sharpe\
           \nReturn %.2f%%,\vol = %.2f%%, \nSharpe = %.2f" %
         (highest_sharpe,
          annualized_ret[highest_sharpe] * 100,
          annualized_vol[highest_sharpe] * 100,
          sharpe_metric[highest_sharpe]
         )
         )

    # Create a DataFrame to store the results for all eigenportfolios
    
    results =\
        (
            pd
            .DataFrame(data = {"Return": annualized_ret,
                               "Vol": annualized_vol,
                               "Sharpe": sharpe_metric}
                      )
        )

    results.dropna(inplace = True)

    results.sort_values(by = ["Sharpe"],
                        ascending = False,
                        inplace = True)

    # Print the top 10 eigenportfolios based on Sharpe ratio

    print(results.head(10)
         )
    
def FindPortfolioVisual():
    
    n_portfolios = len(pca.components_)
    
    annualized_ret = np.array([0.] * n_portfolios)
    
    sharpe_metric = np.array([0.] * n_portfolios)
    
    annualized_vol = np.array([0.] * n_portfolios)
    
    highest_sharpe = 0
    
    stock_tickers = scaled_df.columns.values
    
    n_tickers = len(stock_tickers)
    
    PCs = pca.components_
    
    for i in range(n_portfolios):
        
        pc_w = PCs[i] / sum(PCs[i]
                           )
        
        eigen_prtfi = pd.DataFrame(data = {"weights": pc_w.squeeze()*100}, 
                                   index = stock_tickers)
        
        eigen_prtfi.sort_values(by = ["weights"],
                                ascending = False,
                                inplace = True)
        
        eigen_prti_returns = np.dot(X_Train_Raw.loc[:, eigen_prtfi.index], 
                                    pc_w)
        
        eigen_prti_returns = pd.Series(eigen_prti_returns.squeeze(),
                                       index = X_Train_Raw.index)
        
        er, vol, sharpe = calculate_sharpe_ratio(eigen_prti_returns)
        
        annualized_ret[i] = er
        
        annualized_vol[i] = vol
        
        sharpe_metric[i] = sharpe
        
        sharpe_metric = np.nan_to_num(sharpe_metric)
        
    # HOW TO FIND A PORTFOLIO with the HIGHEST Sharpe Ratio
    
    highest_sharpe = np.argmax(sharpe_metric)
    
    print("Our Eigen Portfolio #%d with the highest Sharpe. Return %.2f%%, vol = %.2f%%, Sharpe = %.2f" %
          (highest_sharpe,
           annualized_ret[highest_sharpe]*100,
           annualized_vol[highest_sharpe]*100,
           sharpe_metric[highest_sharpe]
          )
         )
        
    #####
    
    fig, ax = plt.subplots()
    
    fig.set_size_inches(16, 6)
    
    ax.plot(sharpe_metric, 
            linewidth = 2)
    
    ax.set_title("Sharpe Ratio of Eigen-Portfolios")
    
    ax.set_ylabel("Sharpe Ratio")
    
    ax.set_xlabel("Portfolios")
    
    #####
        
    results = pd.DataFrame(data = {"Return": annualized_ret, "Vol": annualized_vol, "Sharpe": sharpe_metric}
                           )
    
    results.dropna(inplace = True)
    
    results.sort_values(by = ["Sharpe"],
                        ascending = False,
                        inplace = True)
    
    print(results.head(15)
         )
    
    plt.show()
    
FindPortfolioVisual()#这里可以看到sharpe ratio最高的三个weight序号

##分段---------------------------------------------------------------------------------------------------------------------------------##
# Yet another gift

def backtest_PCA_porfolios(eigen):

    eigen_prtfi =\
        (
            pd
            .DataFrame(data = {"weights": eigen.squeeze()
                              },
                       index = stock_tickers)
        )

    eigen_prtfi.sort_values(by = ["weights"],
                            ascending = False,
                            inplace = True)

    eigen_prtfi_returns =\
    (
        np
        .dot(X_Test_Raw
             .loc[ : , eigen_prtfi.index],
             eigen)
    )

    eigen_portfolio_returns =\
    (
        pd
        .Series(eigen_prtfi_returns.squeeze(),
                index = X_Test_Raw.index)
    )

    returns, vol, sharpe = calculate_sharpe_ratio(eigen_portfolio_returns)

    print("Our PCA-based Portfolio:\nReturn = %.2f%%\nVolatility = %.2f%%\nSharpe = %.2f"  %
          (returns * 100, vol * 100, sharpe)
         )

    # Compared with what? Equal-weightage Portfolio

    equal_weight_return =\
    (
        X_Test_Raw * (1 / len(pca.components_)
                     )
    ).sum(axis = 1)

    df_plot =\
        (
            pd
            .DataFrame({"ML Portfolio Return": eigen_portfolio_returns,
                        "Equal Weight Index": equal_weight_return},
                      index = X_Test.index
                      )
        )

    (
        np
        .cumprod(df_plot + 1)
        .plot(title = "Returns of the equal weighted index vs. Eigen-Portfolio",
              figsize = [16, 8]
             )
    )

    plt.show()

##分段---------------------------------------------------------------------------------------------------------------------------------##
backtest_PCA_porfolios(eigen = weights[0]
                      )

##分段---------------------------------------------------------------------------------------------------------------------------------##
backtest_PCA_porfolios(eigen = weights[2]
                      )

##分段---------------------------------------------------------------------------------------------------------------------------------##
backtest_PCA_porfolios(eigen = weights[34]
                      )

##分段---------------------------------------------------------------------------------------------------------------------------------##


## PCA 
solution by group9
### <font color = green> run a principal components analysis (PCA) for portfolio management. Begin your analysis with all the above stocks. Make sure to employ the inclusion criterion of less than 30% of missing values.

In [ ]:
nasdaq100_components = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
df1 = nasdaq100_components
ls_symbol = df1['Ticker'].to_list()
df = pdr.get_data_yahoo(ls_symbol, start = dt.datetime(2010,1,1), end = dt.datetime(2019,12,31))[['Adj Close']]
missing_fractions = \
    df \
        .isnull() \
        .mean() \
        .sort_values(ascending = False)
drop_list = \
    sorted(list(missing_fractions
                [missing_fractions > 0.3]
                .index)
           )
len(drop_list)# here we know that We have to delete 16 Tickers down till FANG

##分段---------------------------------------------------------------------------------------------------------------------------------##
df = \
    df \
        .drop(labels= drop_list,
              axis=1)
df.shape[1] == 101 - 16

##分段---------------------------------------------------------------------------------------------------------------------------------##
df = df.fillna(method = "ffill")
# conduct EDA
np.random.seed(231017)
corr = df.corr()
plt.figure(figsize = [20, 20])
plt.title("A Heatmap for Correlation Matrix")
sns \
    .heatmap(corr,
             annot = True,
             cmap = "viridis")
##分段---------------------------------------------------------------------------------------------------------------------------------##

Daily_Linear_Return = pd.DataFrame()
Daily_Linear_Return = \
    (
        df
        .pct_change(1)
    )
Daily_Linear_Return = \
    (
        Daily_Linear_Return[Daily_Linear_Return
        .apply(lambda x:(x - x.mean()
                         ).abs() < (4 * x.std()
                                    )
               )
        .all(1)
        ]
    )
print("No. of Outliers removed =", df.shape[0] - Daily_Linear_Return.shape[0])

##分段---------------------------------------------------------------------------------------------------------------------------------##
scaler = \
    (
        StandardScaler()
        .fit(Daily_Linear_Return)
    )
scaler

#分段---------------------------------------------------------------------------------------------------------------------------------##
scaled_df = \
    (
        pd
        .DataFrame(scaler.fit_transform(Daily_Linear_Return),
                   columns = Daily_Linear_Return.columns,
                   index = Daily_Linear_Return.index)
    )

prop = \
    int(len(scaled_df) * 0.75)

X_Train = scaled_df[    : prop] # First 75% of the data
X_Test  = scaled_df[prop:     ] # Remaining 25% of the data

X_Train_Raw = Daily_Linear_Return[    :prop]
X_Test_Raw  = Daily_Linear_Return[prop:    ]

tickers = \
    (
        scaled_df
        .columns
        .values
    )

pca = PCA()
PCom = pca.fit(X_Train)
PCom

#分段---------------------------------------------------------------------------------------------------------------------------------##
pca.components_

#分段---------------------------------------------------------------------------------------------------------------------------------##
EigenVal = 10
fig, axes = \
    (
        plt
        .subplots(ncols = 2,
                  figsize = [16, 6]
                  )
    )
# Plot on the left panel
Series1 = \
    (
            pd
            .Series(pca
                    .explained_variance_ratio_[ :EigenVal]
                    )
            .sort_values()
            * 100
    )
# Plot on the right panel
Series2 = \
    (
            pd
            .Series(pca
                    .explained_variance_ratio_[ :EigenVal]
                    )
            .cumsum()
            * 100
    )

(
    Series1
    .plot
    .barh(ylim = (0, 9),
          title = "Explained Variance Ratio by Top 10 PCs",
          ax = axes[0]
          )
)

(
    Series2
    .plot(ylim = (0, 100),
          xlim = (0, 9),
          title = "Cumulative Explained Variance by Each PC",
          ax = axes[1]
          )
)

#分段---------------------------------------------------------------------------------------------------------------------------------##
pca.explained_variance_ratio_


(
    pd
    .Series(np
            .cumsum(pca
                    .explained_variance_ratio_)
            )
    .to_frame("Explained Variance")
    .head(EigenVal)
    .style
    .format("{:,.2%}".format)
)


weights = pd.DataFrame()
for i in range(len(pca.components_)):
    weights["weights_{}".format(i)] = pca.components_[i] / sum(pca.components_[i])
weights = weights.values.T
weights


weights[0]
              
    
# Set the number of principal components at 4
NumComp = 4
# Extract the top 4 principal components from the PCA object
# and create a DataFrame with columns named after the original features
topPorts = \
    (
        pd
        .DataFrame(pca.components_[ : NumComp],
                   columns = df.columns)
    )
# Normalize the weights of the top portfolios such that the weights sum up to 1 for each portfolio
# This is done by dividing each weight by the sum of weights for the respective portfolio
eigen_portfolios = \
    (
        topPorts
        .div(topPorts.sum(1),
             axis = 0)
    )
# Rename the index of the eigen_portfolios DataFrame for better readability
eigen_portfolios.index = [f"Portfolio {i}" for i in range(NumComp)
                          ]
# Calculate the square root of the explained variance for each component
# This provides the standard deviation of returns for each eigenportfolio
np.sqrt(pca.explained_variance_)


## returns the eigen portfolios with the specified number of PrinComp
# eigen_portfolios

## return the first one
# eigen_portfolios.iloc[0]

(
    eigen_portfolios
    .T  # Transpose the DataFrame to have portfolios as columns and assets as rows
    .plot
    .bar(subplots = True,
         layout = (int(NumComp), 1),
         legend = False,
         sharey = True,
         figsize = [16, 20],
         ylim = [-1, 1]
         )
)


## Eigenportfolio#0 is not properly plotted due to lower y-values, replot using different scale
(
    eigen_portfolios.iloc[0]
    .T  # Transpose the DataFrame to have portfolios as columns and assets as rows
    .plot
    .bar(subplots = True,
         layout = (int(NumComp), 1),
         legend = False,
         sharey = True,
         figsize = [16, 20],
         ylim = [-0.02, 0.02]
         )
)


plt.figure(figsize=[16, 6]
           )
sns.heatmap(topPorts,
            cmap = "viridis")


n_ports = len(pca.components_)
 # Initialize arrays for annualized return, volatility, and Sharpe ratio of each eigenportfolio
annualized_ret = np.array([0.] * n_ports)
sharpe_metric = np.array([0.] * n_ports)
annualized_vol = np.array([0.] * n_ports)
# Variable to track the index of the eigenportfolio with the highest Sharpe ratio, initialise before each run = 0
highest_sharpe = 0
# Extract stock tickers from the scaled data
stock_tickers = \
    (scaled_df
     .columns
     .values)
stock_tickers


annualized_ret


PCs = pca.components_
pc_w = PCs[84]/sum(PCs[84])
np.sum(pc_w)


pc_w.squeeze()*100


eigen_prtfi = \
    (
        pd
        .DataFrame(data={"weights": pc_w.squeeze() * 100},
                   index=stock_tickers)
    )
eigen_prtfi
## negative weights = short, positive weights = long


eigen_prtfi.sort_values(by = ["weights"],
                        ascending = False,
                        inplace = True)
eigen_prtfi
### GOOGL features No.1 in weights from among the PCAComp. But it is also the last weight. Why is that? One is Class A and one is Class C (non-voting shares), The net is amount right, +05


X_Train_Raw.loc[ : , eigen_prtfi.index]
## slightly less than 4 years worth of complete data points on top PrinComp starting from left


## Generate returns for the Train Set
eigen_prti_returns = \
    (
        np
        .dot(X_Train_Raw.loc[ : , eigen_prtfi.index],
             pc_w)
    )


periods_per_year = 252
n_years = eigen_prti_returns.shape[0] / periods_per_year
n_years


np.prod(1+eigen_prti_returns)


annualized_return = np.power(np.prod(1 + eigen_prti_returns), (1 / n_years)
                             ) - 1


## Build function for calculating Sharpe Ratio on Train set
def calculate_sharpe_ratio(ts_returns, periods_per_year = 252):

    n_years = ts_returns.shape[0] / periods_per_year

    annualized_return = np.power(np.prod(1 + ts_returns), (1 / n_years)
                                 ) -   1

    annualized_vol = ts_returns.std() * np.sqrt(periods_per_year)

    annualized_sharpe = annualized_return / annualized_vol

    return annualized_return, annualized_vol, annualized_sharpe
# Build a function to recommend Top 4 best eigen portfolios/Top 4 PrinComp
def recommend_optimal_portfolio():
    # Number of eigenportfolios or principal components
    n_portfolios = len(pca.components_[:4])
    # Initialize arrays for annualized return, volatility, and Sharpe ratio of each eigenportfolio
    annualized_ret = np.array([0.] * n_portfolios)
    sharpe_metric = np.array([0.] * n_portfolios)
    annualized_vol = np.array([0.] * n_portfolios)
    # Variable to track the index of the eigenportfolio with the highest Sharpe ratio
    highest_sharpe = 0
    # Extract stock tickers from the scaled data
    stock_tickers = \
        (scaled_df
         .columns
         .values)
    n_tickers = len(stock_tickers)
    # Extract principal components
    PCs = pca.components_[:4]
    # Loop through each eigenportfolio
    for i in range(n_portfolios):
        # Normalize the weights of the i-th eigenportfolio
        pc_w = PCs[i] / sum(PCs[i])
        # Create a DataFrame for the eigenportfolio weights
        eigen_prtfi = \
            (
                pd
                .DataFrame(data = {"weights": pc_w.squeeze() * 100},
                           index = stock_tickers)
            )
        # Calculate returns for the eigenportfolio
        eigen_prtfi.sort_values(by = ["weights"],
                                ascending = False,
                                inplace = True)
        eigen_prti_returns = \
            (
                np
                .dot(X_Train_Raw.loc[ : , eigen_prtfi.index],
                     pc_w)
            )
        eigen_prti_returns = \
            (
                pd
                .Series(eigen_prti_returns.squeeze(),
                        index = X_Train_Raw.index)
            )
        # Calculate annualized return, volatility, and Sharpe ratio for the eigenportfolio
        er, vol, sharpe = calculate_sharpe_ratio(eigen_prti_returns)
        # Store the metrics in their respective arrays
        annualized_ret[i] = er
        annualized_vol[i] = vol
        sharpe_metric[i] = sharpe
        # Replace NaN values in Sharpe metric array with zeros
        sharpe_metric = np.nan_to_num(sharpe_metric)
    # Let's find a portfolio with the HIGHEST Sharpe Ratio
    highest_sharpe = np.argmax(sharpe_metric)
    # Print the details of the eigenportfolio with the highest Sharpe ratio
    print("Our Eigen Portfolio #%d with the highest Sharpe\
           \nReturn %.2f%%,\vol = %.2f%%, \nSharpe = %.2f" %
          (highest_sharpe,
           annualized_ret[highest_sharpe] * 100,
           annualized_vol[highest_sharpe] * 100,
           sharpe_metric[highest_sharpe]
           ))
    # Create a DataFrame to store the results for all eigenportfolios
    results = \
        (
            pd
            .DataFrame(data = {"Return": annualized_ret,
                               "Vol": annualized_vol,
                               "Sharpe": sharpe_metric}
                       ))
    results.dropna(inplace = True)
    results.sort_values(by = ["Sharpe"],
                        ascending = False,
                        inplace = True)
    # Print the top 10 eigenportfolios based on Sharpe ratio
    print(results.head(10)
          )
    return 


recommend_optimal_portfolio()


top = 10
n_portfolios = len(pca.components_[:top])
annualized_ret = np.array([0.] * n_portfolios)
sharpe_metric = np.array([0.] * n_portfolios)
annualized_vol = np.array([0.] * n_portfolios)
highest_sharpe = 0
stock_tickers = scaled_df.columns.values
n_tickers = len(stock_tickers)
PCs = pca.components_[:top]
for i in range(n_portfolios):
    pc_w = PCs[i] / sum(PCs[i])
    eigen_prtfi = pd.DataFrame(data = {"weights": pc_w.squeeze()*100},
                                index = stock_tickers)
    eigen_prtfi.sort_values(by = ["weights"],
                            ascending = False,
                            inplace = True)
    eigen_prti_returns = np.dot(X_Train_Raw.loc[:, eigen_prtfi.index],
                                pc_w)
    eigen_prti_returns = pd.Series(eigen_prti_returns.squeeze(),
                                    index = X_Train_Raw.index)
    er, vol, sharpe = calculate_sharpe_ratio(eigen_prti_returns)
    annualized_ret[i] = er
    annualized_vol[i] = vol
    sharpe_metric[i] = sharpe
    sharpe_metric = np.nan_to_num(sharpe_metric)
# HOW TO FIND A PORTFOLIO with the HIGHEST Sharpe Ratio
highest_sharpe = np.argmax(sharpe_metric)
print("From Train Set, the Eigen Portfolio #%d with the highest Sharpe. Return %.2f%%, vol = %.2f%%, Sharpe = %.2f" %
        (highest_sharpe,
        annualized_ret[highest_sharpe]*100,
        annualized_vol[highest_sharpe]*100,
        sharpe_metric[highest_sharpe]
        ))
fig, ax = plt.subplots()
fig.set_size_inches(16, 6)
ax.plot(sharpe_metric,linewidth = 2)
ax.set_title("Sharpe Ratio of Eigen-Portfolios")
ax.set_ylabel("Sharpe Ratio")
ax.set_xlabel("Portfolios")
results = pd.DataFrame(data = {"Return": annualized_ret, "Vol": annualized_vol, "Sharpe": sharpe_metric})
results.dropna(inplace = True)
results.sort_values(by = ["Sharpe"],
                        ascending = False,
                        inplace = True)
print(results.head(15))
plt.show()


print("Portfolio #0")
eigen=weights[0]
eigen_prtfi =pd.DataFrame(data={"weights": eigen.squeeze()},
                       index=stock_tickers)
eigen_prtfi.sort_values(by=["weights"],
                        ascending=False,
                        inplace=True)
eigen_prtfi_returns = np.dot(X_Test_Raw
                             .loc[:, eigen_prtfi.index],
                             eigen)
eigen_portfolio_returns = pd.Series(eigen_prtfi_returns.squeeze(),
                    index=X_Test_Raw.index)
returns, vol, sharpe = calculate_sharpe_ratio(eigen_portfolio_returns)
print("From Test Set, Our PCA-based Portfolio performed as follows:\nReturn = %.2f%%\nVolatility = %.2f%%\nSharpe = %.2f" %
    (returns * 100, vol * 100, sharpe))
# Compared with what? Equal-weightage Portfolio
equal_weight_return = X_Test_Raw * (1 / len(pca.components_)).sum(axis=1)
df_plot = pd.DataFrame({"ML Portfolio Return": eigen_portfolio_returns,
                        "Equal Weight Index": equal_weight_return},
                       index=X_Test.index
                       )
(
    np
    .cumprod(df_plot + 1)
    .plot(title="Returns of the equal weighted index vs. Eigen-Portfolio",
            figsize=[16, 8]
            )
)
plt.show()


In [ ]:
print("Portfolio #5")
eigen=weights[6]
eigen_prtfi =pd.DataFrame(data={"weights": eigen.squeeze()},
                       index=stock_tickers)
eigen_prtfi.sort_values(by=["weights"],
                        ascending=False,
                        inplace=True)
eigen_prtfi_returns = np.dot(X_Test_Raw
                             .loc[:, eigen_prtfi.index],
                             eigen)
eigen_portfolio_returns = pd.Series(eigen_prtfi_returns.squeeze(),
                    index=X_Test_Raw.index)
returns, vol, sharpe = calculate_sharpe_ratio(eigen_portfolio_returns)
print("From Test Set, Our PCA-based Portfolio performed as follows:\nReturn = %.2f%%\nVolatility = %.2f%%\nSharpe = %.2f" %
    (returns * 100, vol * 100, sharpe))
# Compared with what? Equal-weightage Portfolio
equal_weight_return = X_Test_Raw * (1 / len(pca.components_)).sum(axis=1)
df_plot = pd.DataFrame({"ML Portfolio Return": eigen_portfolio_returns,
                        "Equal Weight Index": equal_weight_return},
                       index=X_Test.index
                       )
(
    np
    .cumprod(df_plot + 1)
    .plot(title="Returns of the equal weighted index vs. Eigen-Portfolio",
            figsize=[16, 8]
            )
)
plt.show()


In [ ]:
(
    eigen_portfolios.iloc[0]
    .T  # Transpose the DataFrame to have portfolios as columns and assets as rows
    .plot
    .bar(subplots = True,
         layout = (int(NumComp), 1),
         legend = False,
         sharey = True,
         figsize = [16, 20],
         ylim = [-0.02, 0.02]
         )
)

### <font color = blue> 可恶多剩下的不想搞了，直接看group9的set吧

## Cluster
### <font color = green> Using the 102 tickers below, and what you have learned in class, run the analysis and develop a dendrogram. Make sure to employ the inclusion criterion of less than 30% of missing values.

In [ ]:
nasdaq100_components = pd.read_html(
    "https://en.wikipedia.org/wiki/Nasdaq-100")[4]
TICKERS = nasdaq100_components['Ticker'].tolist()

# here to change the period
start = datetime.date(2019, 10, 24)
end = datetime.date(2023, 10, 25)

stock_data = []
for ticker in TICKERS:
    stock_data.append(pdr.get_data_yahoo(TICKERS, start, end))
nasdaq100_stocks = pd.concat(stock_data, keys=TICKERS, names=["ticker", "date"])
nasdaq100_closing_prices =\
    (
        nasdaq100_stocks[["Adj Close"]]
        .reset_index()
    )
nasdaq100_closing_prices_WIDE =\
    (
        nasdaq100_closing_prices
        .pivot(index="date",
               columns="ticker",
               values="Adj Close")
    )
# show the proportion of missing values in the entire stock column
missing_fractions = \
    nasdaq100_closing_prices_WIDE \
    .isnull() \
    .mean() \
    .sort_values(ascending=False)
missing_fractions# here we have three columns which have more than 30% missing values

##分段---------------------------------------------------------------------------------------------------------------------------------##
# We create a list containing the names of these three columns.
drop_list =\
    sorted(list(missing_fractions
                [missing_fractions > 0.3]
                .index)
           )
# Then we drop them.
nasdaq100_closing_prices_WIDE =\
    nasdaq100_closing_prices_WIDE \
    .drop(labels=drop_list,
          axis=1)
nasdaq100_closing_prices_WIDE.shape[1] == 101 - 3# as we have 101 columns in total, and three columns which have more than 30% missing values

##分段---------------------------------------------------------------------------------------------------------------------------------##
nasdaq100_closing_prices_WIDE = nasdaq100_closing_prices_WIDE.fillna(
    method="ffill")
returns = nasdaq100_closing_prices_WIDE.pct_change().dropna()
from scipy.cluster.hierarchy import dendrogram, linkage, ward
returns_corr = returns.corr()# correlation between columns
dist = ((1 - returns_corr) / 2.0) ** 0.5
# Then we can easily calculate the linkage, which is the basis for us to draw Dendograms.
link =linkage(dist,"ward")

plt.figure(figsize=(18, 10))
plt.title("Stocks Dendograms")
dendrogram(link, labels=dist.index)
plt.show()

##分段---------------------------------------------------------------------------------------------------------------------------------##
link[0]
##分段---------------------------------------------------------------------------------------------------------------------------------##
link_up = link[np.argsort(link[:, 2])]
link_up[0]# here we get the reault 41 and 42
#分段---------------------------------------------------------------------------------------------------------------------------------##
stock_mapping = {index: stock_name for
                 index,stock_name
                 in enumerate(dist.index)}
print(
    f"{stock_mapping[41]} and {stock_mapping[42]} is the most corelated pair of stocks.")

##分段---------------------------------------------------------------------------------------------------------------------------------##
link_down = link_stocks[np.argsort(link_stocks[:, 2])[::-1]]
link_down[0]# here we get the reault 51 and 83
#分段---------------------------------------------------------------------------------------------------------------------------------##
print(f"{stock_mapping[51]} and {stock_mapping[83]} is the least correlated pair of stocks.")

##分段---------------------------------------------------------------------------------------------------------------------------------##
plt.figure(figsize=(18, 10))
plt.title("Stocks Dendograms")
dendrogram(link, labels=dist.index)
plt.show()

##分段---------------------------------------------------------------------------------------------------------------------------------##
print(f"{stock_mapping[41]} and {stock_mapping[42]} is the most corelated pair of stocks.")
print(f"{stock_mapping[51]} and {stock_mapping[83]} is the least correlated pair of stocks.")


